In [1]:
pip install tensorflow numpy matplotlib scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [4]:
# Configuration Parameters
import os

# Paths
TRAIN_DIR = 'dataset/train'
VALIDATION_DIR = 'dataset/validation'
MODEL_SAVE_PATH = 'final_deepfake_detector.h5'

# Image settings
IMAGE_SIZE = (299, 299)  # Xception's default input size
BATCH_SIZE = 32

# Training settings
INITIAL_EPOCHS = 20
FINE_TUNE_EPOCHS = 20
TOTAL_EPOCHS = INITIAL_EPOCHS + FINE_TUNE_EPOCHS
FINE_TUNE_AT = 100  # Layer from which to fine-tune

# Learning rates
INITIAL_LR = 1e-4
FINE_TUNE_LR = 1e-5


In [5]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

def build_model(image_size=IMAGE_SIZE):
    # Load the Xception model without the top layers
    base_model = Xception(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))
    
    # Freeze the base model
    base_model.trainable = False
    
    # Add custom top layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)  # Regularization
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)  # Binary classification
    
    # Construct the complete model
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=INITIAL_LR),
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
    
    return model

model = build_model()
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                           

 block4_sepconv1_bn (BatchNorma  (None, 37, 37, 728)  2912       ['block4_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block4_sepconv2_act (Activatio  (None, 37, 37, 728)  0          ['block4_sepconv1_bn[0][0]']     
 n)                                                                                               
                                                                                                  
 block4_sepconv2 (SeparableConv  (None, 37, 37, 728)  536536     ['block4_sepconv2_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block4_sepconv2_bn (BatchNorma  (None, 37, 37, 728)  2912       ['block4_sepconv2[0][0]']        
 lization)

 n)                                                                                               
                                                                                                  
 block7_sepconv1 (SeparableConv  (None, 19, 19, 728)  536536     ['block7_sepconv1_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block7_sepconv1_bn (BatchNorma  (None, 19, 19, 728)  2912       ['block7_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block7_sepconv2_act (Activatio  (None, 19, 19, 728)  0          ['block7_sepconv1_bn[0][0]']     
 n)                                                                                               
          

 block9_sepconv3_bn (BatchNorma  (None, 19, 19, 728)  2912       ['block9_sepconv3[0][0]']        
 lization)                                                                                        
                                                                                                  
 add_19 (Add)                   (None, 19, 19, 728)  0           ['block9_sepconv3_bn[0][0]',     
                                                                  'add_18[0][0]']                 
                                                                                                  
 block10_sepconv1_act (Activati  (None, 19, 19, 728)  0          ['add_19[0][0]']                 
 on)                                                                                              
                                                                                                  
 block10_sepconv1 (SeparableCon  (None, 19, 19, 728)  536536     ['block10_sepconv1_act[0][0]']   
 v2D)     

                                                                                                  
 block12_sepconv3_act (Activati  (None, 19, 19, 728)  0          ['block12_sepconv2_bn[0][0]']    
 on)                                                                                              
                                                                                                  
 block12_sepconv3 (SeparableCon  (None, 19, 19, 728)  536536     ['block12_sepconv3_act[0][0]']   
 v2D)                                                                                             
                                                                                                  
 block12_sepconv3_bn (BatchNorm  (None, 19, 19, 728)  2912       ['block12_sepconv3[0][0]']       
 alization)                                                                                       
                                                                                                  
 add_22 (A

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_data_generators(train_dir=TRAIN_DIR, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE):
    # Data augmentation for the training set
    train_datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        horizontal_flip=True,
        rotation_range=20,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        fill_mode="nearest",
        validation_split=0.2  # 80-20 split
    )
    
    # Training generator
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='training',
        shuffle=True
    )
    
    # Validation generator
    validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='validation',
        shuffle=False
    )
    
    return train_generator, validation_generator

train_generator, validation_generator = create_data_generators()


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'dataset/train'

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

def train_model(model, train_gen, val_gen, save_path=MODEL_SAVE_PATH, initial_epochs=INITIAL_EPOCHS):
    # Define callbacks
    checkpoint = ModelCheckpoint(save_path,
                                 monitor='val_auc',
                                 verbose=1,
                                 save_best_only=True,
                                 mode='max')
    
    early_stop = EarlyStopping(monitor='val_auc',
                               patience=10,
                               verbose=1,
                               restore_best_weights=True)
    
    callbacks_list = [checkpoint, early_stop]
    
    # Train the model
    history = model.fit(
        train_gen,
        steps_per_epoch=train_gen.samples // train_gen.batch_size,
        epochs=initial_epochs,
        validation_data=val_gen,
        validation_steps=val_gen.samples // val_gen.batch_size,
        callbacks=callbacks_list
    )
    
    return history

history = train_model(model, train_generator, validation_generator)


NameError: name 'train_generator' is not defined

In [8]:
# Save the final model
model.save(MODEL_SAVE_PATH)
print(f"Model saved to {MODEL_SAVE_PATH}")


Model saved to final_deepfake_detector.h5


In [9]:
def load_model_from_path(model_path=MODEL_SAVE_PATH):
    loaded_model = tf.keras.models.load_model(model_path)
    print(f"Model loaded from {model_path}")
    return loaded_model

# Example usage:
# loaded_model = load_model_from_path()


In [10]:
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

def predict_image(model, img_path, image_size=IMAGE_SIZE):
    """
    Predicts whether an image is real or fake.

    Parameters:
    - model: Trained Keras model.
    - img_path: Path to the image file.
    - image_size: Target size for the image.

    Returns:
    - label: 'Fake' or 'Real'.
    - confidence: Confidence score.
    """
    img = image.load_img(img_path, target_size=image_size)
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Create batch axis
    img_array /= 255.0  # Rescale
    
    prediction = model.predict(img_array)[0][0]
    label = 'Fake' if prediction < 0.5 else 'Real'
    confidence = prediction if prediction >= 0.5 else 1 - prediction
    print(f"Prediction: {label} ({confidence*100:.2f}% confidence)")
    return label, confidence

# Example usage:
# label, confidence = predict_image(loaded_model, 'path_to_new_image.jpg')


In [11]:
def main(action='train', image_path=None):
    """
    Main function to train, save, load, and predict using the deepfake detection model.

    Parameters:
    - action: 'train' or 'predict'
    - image_path: Path to the image file for prediction (required if action is 'predict')
    """
    if action == 'train':
        print("Building the model...")
        model = build_model()
        
        print("Creating data generators...")
        train_gen, val_gen = create_data_generators()
        
        print("Training the model...")
        history = train_model(model, train_gen, val_gen)
        
        print("Saving the model...")
        model.save(MODEL_SAVE_PATH)
        print(f"Model saved to {MODEL_SAVE_PATH}")
        
        print("Training completed.")
        
    elif action == 'predict':
        if image_path is None:
            print("Please provide the path to the image for prediction.")
            return
        
        print("Loading the model...")
        loaded_model = load_model_from_path()
        
        print(f"Predicting the image: {image_path}")
        label, confidence = predict_image(loaded_model, image_path)
        
    else:
        print("Invalid action. Please choose 'train' or 'predict'.")


In [12]:
main(action='train')



Building the model...


Creating data generators...


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'dataset/train'

In [13]:
image_path = 'path_to_new_image.jpg'
main(action='predict', image_path=image_path)


Loading the model...
Model loaded from final_deepfake_detector.h5
Predicting the image: path_to_new_image.jpg


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_new_image.jpg'